In [ ]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import catalog as c
from databricks.sdk.service.workspace import (
    ScopeBackendType, AzureKeyVaultSecretScopeMetadata
)
from databricks.sdk.service.settings import TokenAccessControlRequest, TokenPermissionLevel

import yaml

with open("../config.yml") as f:
    cfg = yaml.safe_load(f)

w = WorkspaceClient(host=cfg["workspace_url"], auth_type = 'azure-cli')

# storage x access connector
access_connector_id = f"/subscriptions/{cfg["subscription_id"]}/resourceGroups/rg-medisure-dev/providers/Microsoft.Databricks/accessConnectors/dac-medisure-dev"
sa_name = "samedisuredev"

# secrets x keyvault
SCOPE_NAME = "medisure-dev"
KV_RESOURCE_ID = f"/subscriptions/{cfg["subscription_id"]}/resourceGroups/rg-medisure-dev/providers/Microsoft.KeyVault/vaults/kv-medisure-dev"
KV_DNS_NAME    = "https://kv-medisure-dev.vault.azure.net/"

# databricks storage credentials
cred_name = "cred_medisure_dev"

# external location
ext_name   = "ext_medisure_landing_dev"
ext_delta_files_name = "ext_medisure_delta_files_dev"

#catalog
catalog   = "medisure_dev"

# ABFSS URLs
abfss_url =  f"abfss://landing@{sa_name}.dfs.core.windows.net/"
managed_delta_files_url =  f"abfss://managed-delta-files@{sa_name}.dfs.core.windows.net/"

# service principal
service_principal_name = "medisure dev - workflows"

In [ ]:
# --- Storage Credential (via Access Connector) ---
try:
    w.storage_credentials.get(name=cred_name)
    print(f"Storage credential '{cred_name}' already exists")
except Exception:
    w.storage_credentials.create(
        name=cred_name,
        azure_managed_identity=c.AzureManagedIdentity(
            access_connector_id=access_connector_id
        ),
        comment="ADLS Gen2 via Azure Databricks Access Connector (dev)"
    )
    print(f"Created storage credential '{cred_name}'")

In [ ]:
# --- External Locations  ---

try:
    w.external_locations.get(name=ext_name)
    print(f"External location '{ext_name}' already exists")
except Exception:
    w.external_locations.create(
        name=ext_name,
        url=abfss_url,
        credential_name=cred_name,
        comment=f"{ext_name} location"
    )
    print(f"Created external location '{ext_name}' -> {abfss_url}")

try:
    w.external_locations.get(name=ext_delta_files_name)
    print(f"External location '{ext_delta_files_name}' already exists")
except Exception:
    w.external_locations.create(
        name=ext_delta_files_name,
        url=managed_delta_files_url,
        credential_name=cred_name,
        comment=f"{ext_delta_files_name} location"
    )
    print(f"Created external location '{ext_delta_files_name}' -> {managed_delta_files_url}")

In [ ]:
# --- Catalog (with managed location) ---
managed_root = managed_delta_files_url

try:
    w.catalogs.get(name=catalog)
    print(f"Catalog '{catalog}' already exists")
except Exception:
    w.catalogs.create(
        name=catalog,
        comment="MediSure dev catalog",
        storage_root=managed_root,     # makes catalog 'managed' at this path
    )
    print(f"Created catalog '{catalog}' with storage_root {managed_root}")

In [ ]:
# # --- Schemas: bronze/silver/gold --- to be created as part of application
# for schema in ["bronze", "silver", "gold"]:
#     full = f"{catalog}.{schema}"
#     try:
#         w.schemas.get(full_name=full)
#         print(f"Schema '{full}' already exists")
#     except Exception:
#         w.schemas.create(name=schema, catalog_name=catalog, comment=f"{schema} layer")
#         print(f"Created schema '{full}'")

In [ ]:
# --- Secrets: azure key vault ---
existing = [s.name for s in w.secrets.list_scopes()]
if SCOPE_NAME in existing:
    print(f"Scope '{SCOPE_NAME}' already exists.")
else:
    w.secrets.create_scope(
        scope=SCOPE_NAME,
        scope_backend_type=ScopeBackendType.AZURE_KEYVAULT,
        backend_azure_keyvault=AzureKeyVaultSecretScopeMetadata(
            resource_id=KV_RESOURCE_ID,
            dns_name=KV_DNS_NAME
        )
    )
    print(f"Created AKV-backed scope '{SCOPE_NAME}'.")

In [ ]:
# --- Service Principal for Workflows ---
sp = w.service_principals.create(display_name=f"{service_principal_name}")
print("created service principal:", sp.application_id, sp.display_name)

# grant access to catalog; service principals will create schema and all objects so by default it becomes owner

w.grants.update(
    securable_type="CATALOG",
    full_name=catalog,
    changes=[
        c.PermissionsChange(
            principal=sp.application_id,
            add=[c.Privilege.USE_CATALOG, c.Privilege.CREATE_SCHEMA]
        ),
    ],
)

In [ ]:
# --- Service Principal for Deployment ---
sp = w.service_principals.create(display_name="github deploy - dev")
print("created service principal:", sp.application_id, sp.display_name)

In [ ]:
# create token for self to open up token permission settings; without this we cannot grant access to tokens, because no tokens exist yet
w.tokens.create(comment="initial token",lifetime_seconds=60)

# allow access to tokens for service principal
access_control_list = [
    TokenAccessControlRequest(
        service_principal_name=sp.application_id,
        permission_level=TokenPermissionLevel.CAN_USE,  # Use the enum value
    )
]

token_permissions = w.token_management.update_permissions(
    access_control_list=access_control_list
)
token_permissions.as_dict()

In [ ]:
# does not work on Azure, it is possibly blocked
# # Create a PAT on behalf of the SP
# obo = w.token_management.create_obo_token(
#     application_id=sp.application_id,          # the SP’s application (client) ID
#     comment="github-deploy dev",
#     lifetime_seconds=60 * 60 * 24 * 30         # 30 days; adjust to your policy
# )

# print("TOKEN_ID:", obo.token_info.token_id)
# print("TOKEN_VALUE (copy now):", obo.token_value)

In [ ]:
# create client secret as service principal
create_secrets_response = w.service_principal_secrets_proxy.create(service_principal_id=sp.id)

# login as service principal
w_as_sp = WorkspaceClient(host=cfg["workspace_url"],
                client_id = sp.application_id,
                client_secret = create_secrets_response.secret,
                auth_type='oauth-m2m'
                )

# create token as service principal
create_token_response = w_as_sp.tokens.create(
    comment = sp.display_name,
    lifetime_seconds=-1)

delete_secrets_response = w.service_principal_secrets_proxy.delete(
    service_principal_id=sp.id, 
    secret_id=create_secrets_response.id)

print(create_secrets_response)
print(create_token_response)